In [ ]:
# uncomment the following if you had not installed the package
# import sys
# sys.path.append('/home/khan/Documents/developments/pyIntertidalDEM')

from pyintdem.core import RGB
from pyintdem.data import Database
from pathlib import Path
import logging

from pyintdem.workflows.khan2019 import process_database

In [3]:
logging.basicConfig()
logging.basicConfig(level=logging.INFO,
                    filename='03_extract_shoreline.log',
                    filemode='w')
logging.captureWarnings(True)

In [4]:
datadir = Path('/run/media/khan/Workbench/Developments/Bengal/Data')
maskdir = Path('/run/media/khan/Workbench/Developments/Bengal/Masks')
shorelinedir = Path('/run/media/khan/Workbench/Developments/Bengal/Shorelines')

In [5]:
database = Database(datadir)
zones = database.tiles
database.filter()

{'T46QBK': [{'filetype': 'theia',
   'fpath': '/run/media/khan/Workbench/Developments/Bengal/Data/T46QBK/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8.zip',
   'mission': 'SENTINEL2A',
   'product': 'L2A',
   'version': 'V1-8',
   'time': Timestamp('2016-01-18 04:43:55.092000'),
   'tile': 'T46QBK',
   'bands': {'B11': '/vsizip//run/media/khan/Workbench/Developments/Bengal/Data/T46QBK/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8.zip/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8_FRE_B11.tif',
    'B4': '/vsizip//run/media/khan/Workbench/Developments/Bengal/Data/T46QBK/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8.zip/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8_FRE_B4.tif',
    'B6': '/vsizip//run/media/khan/Workbench/Developments/Bengal/Data/T46QBK/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8.zip/SENTINEL2A_20160118-044355-092_L2A_T46QBK_D_V1-8/SENTINEL2A_20160118-0443

In [6]:
# Computation
nhue = 0.5 # Thresholding parameter for Hue band
nvalue = 3.0 # Thresholding parameter for Value band

waterblob = 10000 # Remove water blob smaller than 10000 pixels
landblob = 10000 # Remove land blobs smaller than 10000 pixels

process_database(database=database,
                 out_dir=shorelinedir,
                 mask_dir=maskdir,
                 nhue=nhue, nvalue=3.0,
                 waterblob=waterblob, landblob=landblob,
                 savetifs=False, saveplots=True,
                 clip_kw=None, recompute=False)
